In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
from torch import nn

In [3]:
import pickle

In [4]:
with open('files/word_to_int.p', 'rb') as f:
    word_to_int = pickle.load(f)

In [5]:
word_to_int

{',': 0,
 '.': 1,
 'the': 2,
 '"': 3,
 'and': 4,
 'to': 5,
 'of': 6,
 'he': 7,
 "'": 8,
 'a': 9,
 'in': 10,
 'that': 11,
 'was': 12,
 'his': 13,
 'her': 14,
 'it': 15,
 'i': 16,
 'she': 17,
 'had': 18,
 'with': 19,
 'not': 20,
 's': 21,
 'you': 22,
 'him': 23,
 'but': 24,
 'at': 25,
 'said': 26,
 'for': 27,
 'as': 28,
 '?': 29,
 'on': 30,
 'all': 31,
 'what': 32,
 'be': 33,
 '!': 34,
 ';': 35,
 'levin': 36,
 'so': 37,
 'is': 38,
 't': 39,
 'this': 40,
 'up': 41,
 'from': 42,
 'have': 43,
 'one': 44,
 'they': 45,
 'were': 46,
 'there': 47,
 'no': 48,
 'me': 49,
 'by': 50,
 'which': 51,
 'been': 52,
 'would': 53,
 'out': 54,
 'could': 55,
 'when': 56,
 'did': 57,
 'now': 58,
 'vronsky': 59,
 'them': 60,
 'do': 61,
 'anna': 62,
 'who': 63,
 'how': 64,
 'into': 65,
 'my': 66,
 'an': 67,
 'about': 68,
 'more': 69,
 'their': 70,
 '_': 71,
 'go': 72,
 'know': 73,
 'kitty': 74,
 'well': 75,
 'come': 76,
 'went': 77,
 'only': 78,
 'very': 79,
 'can': 80,
 'if': 81,
 'alexey': 82,
 'are': 83,
 '

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [7]:
device

'mps'

In [8]:
class WordLSTM(nn.Module):
    def __init__(self, n_embed=128, n_layers=3, drop_prob=.2):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_embed = n_embed
        vocab_size = len(word_to_int)
        self.embedding = nn.Embedding(vocab_size, n_embed)
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_embed, num_layers=n_layers, dropout=drop_prob, batch_first=True)
        self.fc = nn.Linear(n_embed, vocab_size)

    def forward(self, x, hc):
        embed = self.embedding(x)
        x, hc = self.lstm(embed, hc)
        x = self.fc(x)
        return x, hc

    def init_hidden(self, n_seqs):
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_embed).zero_().to(device),
                weight.new(self.n_layers, n_seqs, self.n_embed).zero_().to(device))

In [9]:
states = torch.load('files/wordLSTM.pth', map_location=device)

In [10]:
lstm = WordLSTM().to(device)

In [11]:
lstm.load_state_dict(states)

<All keys matched successfully>

In [12]:
int_to_word={v:k for k,v in word_to_int.items()}

In [13]:
seq_length = 100
def sample(model, prompt, length=200):
    model.eval() 
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    length = length - len(text)
    for _ in range(length):
        if len(text)  <= seq_length:
            x = torch.tensor([[word_to_int[w] for w in text]])
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-seq_length:]]])
        inputs = x.to(device)
        outputs, hc = model(inputs, hc)
        logits = outputs[0][-1]
        p = nn.functional.softmax(logits, dim=0).detach().cpu().numpy()
        idx = np.random.choice(len(logits), p=p)
        text.append(int_to_word[idx])
    text=" ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text=text.replace(f" {m}", f"{m} ")
    text=text.replace('" ', '"')
    text=text.replace("' ", "'")
    text=text.replace('" ', '"')
    text=text.replace("' ", "'")
    return text

In [14]:
torch.manual_seed(42)
np.random.seed(42)
sample(lstm, "Anna and the prince")

'anna and the prince did not forget what he had not spoken.  when the softening barrier was not so long as he had talked to his brother,  all the hopelessness of the impression.  "official tail,  a man who had tried him,  though he had been able to get across his charge and locked close,  and the light round the snow was in the light of the altar villa.  the article in law levin was first more precious than it was to him so that if it was most easy as it would be as the same.  this was now perfectly interested.  when he had got up close out into the sledge,  but it was locked in the light window with their one grass,  and in the band of the leaves of his projects,  and all the same stupid woman,  and really,  and i swung his arms round that thinking of bed.  a little box with the two boys were with the point of a gleam of filling the boy,  noiselessly signed the bottom of his mouth,  and answering them took the red'

In [20]:
def generate(model, prompt, top_k=None, length=200, temperature=1):
    model.eval() 
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    length = length - len(text)
    for _ in range(length):
        if len(text)  <= seq_length:
            x = torch.tensor([[word_to_int[w] for w in text]])
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-seq_length:]]])
        inputs = x.to(device)
        outputs, hc = model(inputs, hc)
        logits = outputs[0][-1]
        logits /= temperature
        p = nn.functional.softmax(logits, dim=0).detach().cpu()
        if top_k is None:
            idx = np.random.choice(len(logits), p=p.numpy())
        else:
            ps, tops = p.topk(top_k)
            ps /= ps.sum()
            idx = np.random.choice(tops, p=ps.numpy())
        text.append(int_to_word[idx])
    text=" ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text=text.replace(f" {m}", f"{m} ")
    text=text.replace('" ', '"')
    text=text.replace("' ", "'")
    text=text.replace('" ', '"')
    text=text.replace("' ", "'")
    return text

In [21]:
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(lstm, prompt, top_k=None, length=len(prompt.split(" "))+1, temperature=1))

i'm not going to see you
i'm not going to see those
i'm not going to see me
i'm not going to see you
i'm not going to see her
i'm not going to see her
i'm not going to see the
i'm not going to see my
i'm not going to see you
i'm not going to see me


In [22]:
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(lstm, prompt, top_k=3, length=len(prompt.split(" "))+1, temperature=0.5))

i'm not going to see you
i'm not going to see the
i'm not going to see her
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see her
i'm not going to see you
i'm not going to see her


In [23]:
torch.manual_seed(42)
np.random.seed(42)
print(generate(lstm, prompt='Anna and the prince', top_k=3, temperature=0.5))

anna and the prince had no milk.  but,  "answered levin,  and he stopped.  "i've been skating to look at you all the harrows,  and i'm glad. . .  ""no,  i'm going to the country.  ""no,  it's not a nice fellow.  ""yes,  sir.  ""well,  what do you think about it?  ""why,  what's the matter?  ""yes,  yes,  "answered levin,  smiling,  and he went into the hall.  "yes,  i'll come for him and go away,  "he said,  looking at the crumpled front of his shirt.  "i have not come to see him,  "she said,  and she went out.  "i'm very glad,  "she said,  with a slight bow to the ambassador's hand.  "i'll go to the door.  "she looked at her watch,  and she did not know what to say


In [24]:
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(lstm, prompt, top_k=None, length=len(prompt.split(" "))+1, temperature=2))

i'm not going to see them
i'm not going to see scarlatina
i'm not going to see behind
i'm not going to see us
i'm not going to see it
i'm not going to see it
i'm not going to see a
i'm not going to see misery
i'm not going to see another
i'm not going to see seryozha


In [27]:
torch.manual_seed(42)
np.random.seed(42)
print(generate(lstm, prompt='Anna and the prince', top_k=None, temperature=2))

anna and the prince took sheaves covered suddenly people.  "pyotr marya borissovna,  propped mihail though her son will seen how much evening her husband;  if tomorrow she liked great time too.  "adopted heavens details for it women from this terrible,  admitting this touching all everything ill with flirtation shame consolation altogether:  ivan only all the circle with her honorable carriage in its house dress,  beethoven ashamed had the conversations raised mihailov stay of close i taste work?  "on new farming show ivan nothing.  hat yesterday if interested understand every hundred of two with six thousand roubles according to women living over a thousand:  snetkov possibly try disagreeable schools with stake old glory mysterious one have people some moral conclusion,  got down and then their wreath.  darya alexandrovna thought inwardly peaceful with varenka out of the listen from and understand presented she was impossible anguish.  simply satisfied with staying after presence came